# TVA sur les biens alimentaires

In [1]:
from __future__ import division

import numpy as np
import os
import pandas as pd
import pkg_resources
import seaborn

from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.build_survey_data.calibration_aliss import get_adjusted_input_data_frame


In [2]:
year = 2014
data_year = 2011
reform_key = 'aliss_sante'
scenario_kwargs = dict(year = year, data_year = 2011, reform_key = reform_key)
survey_scenario = SurveyScenario.create(**scenario_kwargs)
# 

In [3]:
adjusted_scenario_kwargs = dict(scenario_kwargs)
adjusted_scenario_kwargs.update(dict(
    data_year = None,
    input_data_frame = get_adjusted_input_data_frame(reform_key = reform_key[6:])
    ))
adjusted_survey_scenario = SurveyScenario.create(**adjusted_scenario_kwargs) 

('kantar_expenditures_total:', 52.035641343999998)
('bdf_expenditures_total: ', 52.218515455999999)
('population_kantar_total: ', 11968.229858398438)
('bdf_expenditures_total: ', 126.62905879583185)

/home/benjello/openfisca/openfisca-france-indirect-taxation/openfisca_france_indirect_taxation/build_survey_data/calibration_aliss.py:118: FutureWarning: eval expressions containing an assignment currentlydefault to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  input_data_frame.eval("age = 0 + (agepr > 30) + (agepr > 45) + (agepr > 60)",
/home/benjello/openfisca/openfisca-survey-manager/openfisca_survey_manager/statshelpers.py:122: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  ret[and_(data >= lower, data < upper)] = labels[i]
/home/benjello/openfisca/openfisca-france-indirect-taxation/openfisca_france_indirect_taxation/build_survey_data/calibration_aliss.py:506: FutureWarning: eval expressions containing an assignment currentlydefault to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  'adjusted_bdf_budget_share = kan

In [16]:
alimentation_domicile_hors_alcool = [
    "depenses_ht_{}".format(key) for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'poste_01')
    ]
alimentation_domicile = alimentation_domicile_hors_alcool + [
    'depenses_biere',
    'depenses_vin',
    'depenses_alcools_forts'
    ]
depenses_ht_tvas = [
    "depenses_ht_{}".format(key) for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'tva_taux_')
    ]
tvas = [
    key for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'tva_taux_')
    ] + ['tva_total']


In [17]:
variables = alimentation_domicile + [
    'poste_agrege_01',
    'poste_agrege_02', 
    ] + depenses_ht_tvas + tvas

In [33]:
pivot_table = pd.DataFrame()
reference_pivot_table = pd.DataFrame()
adjusted_pivot_table = pd.DataFrame()
aggregates = dict()
reference_aggregates = dict()
adjusted_aggregates = dict()

for variable in variables:
    pivot_table = pd.concat([
        pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    reference_pivot_table = pd.concat([
        reference_pivot_table.fillna(0),
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'],
                                           reference = True)
        ])
    adjusted_pivot_table = pd.concat([
        adjusted_pivot_table,
        adjusted_survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    aggregates[variable] = survey_scenario.compute_aggregate(variable) / 1e9
    reference_aggregates[variable] = (
        survey_scenario.compute_aggregate(variable, reference = True) / 1e9
        if np.isfinite(survey_scenario.compute_aggregate(variable, reference = True)) else 0
        )
    adjusted_aggregates[variable] = adjusted_survey_scenario.compute_aggregate(variable) / 1e9
    


In [34]:
assert sum(aggregates[aliment] - reference_aggregates[aliment] for aliment in alimentation_domicile) == 0

In [35]:
assert aggregates['poste_agrege_02'] - reference_aggregates['poste_agrege_02'] < 1e-7

In [36]:
for key in sorted(aggregates.keys()):
    if round(aggregates[key] - reference_aggregates[key], 3) != 0:
         print key, ': ', round(aggregates[key], 3), ' - ', round(reference_aggregates[key], 3), ' = ', round(aggregates[key] - reference_aggregates[key], 3)

depenses_ht_tva_taux_01_1_1_1_1 :  13.376  -  0.0  =  13.376
depenses_ht_tva_taux_01_1_1_3_3 :  3.73  -  0.0  =  3.73
depenses_ht_tva_taux_01_1_1_4_1 :  1.488  -  0.0  =  1.488
depenses_ht_tva_taux_01_1_1_4_2 :  1.615  -  0.0  =  1.615
depenses_ht_tva_taux_01_1_1_4_3 :  0.851  -  0.0  =  0.851
depenses_ht_tva_taux_01_1_4_1_1_c :  0.113  -  0.0  =  0.113
depenses_ht_tva_taux_01_1_4_2_1 :  3.361  -  0.0  =  3.361
depenses_ht_tva_taux_01_1_4_2_2 :  2.055  -  0.0  =  2.055
depenses_ht_tva_taux_01_1_4_3_1 :  7.091  -  0.0  =  7.091
depenses_ht_tva_taux_01_1_7_1_1_a :  1.53  -  0.0  =  1.53
depenses_ht_tva_taux_01_1_7_1_1_c :  2.104  -  0.0  =  2.104
depenses_ht_tva_taux_01_1_7_2_1_c :  1.571  -  0.0  =  1.571
depenses_ht_tva_taux_01_1_7_2_2 :  1.982  -  0.0  =  1.982
depenses_ht_tva_taux_01_1_7_2_3 :  0.945  -  0.0  =  0.945
depenses_ht_tva_taux_01_1_8_1_2 :  1.165  -  0.0  =  1.165
depenses_ht_tva_taux_01_1_9_1_1_a :  1.444  -  0.0  =  1.444
depenses_ht_tva_taux_01_1_9_3_2 :  0.928  -  0.0

In [37]:
for key in sorted(aggregates.keys()):
    if round(adjusted_aggregates[key] - aggregates[key], 3) != 0:
         print key, ': ', round(adjusted_aggregates[key], 3), ' - ', round(aggregates[key], 3), ' = ', round(adjusted_aggregates[key] - aggregates[key], 3)

depenses_alcools_forts :  3.082  -  2.932  =  0.15
depenses_biere :  1.043  -  1.301  =  -0.258
depenses_ht_poste_01_10_1 :  2.868  -  2.924  =  -0.056
depenses_ht_poste_01_10_2 :  1.401  -  1.427  =  -0.025
depenses_ht_poste_01_1_1_1_1 :  12.493  -  13.376  =  -0.883
depenses_ht_poste_01_1_1_3_3 :  3.563  -  3.73  =  -0.167
depenses_ht_poste_01_1_1_4_1 :  1.379  -  1.488  =  -0.109
depenses_ht_poste_01_1_1_4_2 :  1.559  -  1.615  =  -0.056
depenses_ht_poste_01_1_1_4_3 :  0.752  -  0.851  =  -0.099
depenses_ht_poste_01_1_2_1_1 :  7.068  -  7.457  =  -0.389
depenses_ht_poste_01_1_2_3_1 :  1.07  -  1.075  =  -0.005
depenses_ht_poste_01_1_2_4_1 :  2.162  -  2.176  =  -0.014
depenses_ht_poste_01_1_2_4_2 :  8.077  -  8.374  =  -0.297
depenses_ht_poste_01_1_2_5_1 :  4.522  -  4.766  =  -0.244
depenses_ht_poste_01_1_2_6_1 :  0.479  -  0.471  =  0.009
depenses_ht_poste_01_1_2_6_3 :  2.915  -  2.943  =  -0.028
depenses_ht_poste_01_1_3_1_1 :  2.65  -  2.841  =  -0.191
depenses_ht_poste_01_1_3_1_

In [38]:
from openfisca_france_indirect_taxation.model.consommation.postes_coicop import get_legislation_data_frames
categories_fiscales_data_frame, codes_coicop_data_frame = get_legislation_data_frames()

In [41]:
aggregates['poste_agrege_01'] - reference_aggregates['poste_agrege_01']

0.72861671211165913

In [43]:
adjusted_aggregates['poste_agrege_01'] - aggregates['poste_agrege_01']

-4.4142565373101661

In [25]:
pivot_table - reference_pivot_table

niveau_vie_decile,1,2,3,4,5,6,7,8,9,10
depenses_ht_poste_01_1_1_1_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_1_3_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_1_4_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_1_4_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_1_4_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_1_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_3_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_4_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_4_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_5_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
adjusted_pivot_table - pivot_table

niveau_vie_decile,1,2,3,4,5,6,7,8,9,10
depenses_ht_poste_01_1_1_1_1,-18.218033,-17.342730,-15.140440,-9.688692,-19.038821,-12.963157,-27.815562,-19.771139,-28.366650,-25.551863
depenses_ht_poste_01_1_1_3_3,4.042053,-6.017961,-2.150764,-0.377008,-0.735014,-3.991449,-1.588538,-6.806316,-3.535960,-4.342269
depenses_ht_poste_01_1_1_4_1,-0.682333,-4.986062,-0.285780,-2.146674,0.363717,-4.335628,-1.022939,-4.520425,-3.067295,-4.575596
depenses_ht_poste_01_1_1_4_2,0.692460,-1.253358,0.009300,1.223270,-0.104166,-1.714872,-0.489341,-1.054938,-0.714543,-1.819802
depenses_ht_poste_01_1_1_4_3,-14.401392,-4.527807,-2.316072,-0.463000,-0.935964,-1.752303,-1.075052,-0.698690,-0.856457,-0.720570
depenses_ht_poste_01_1_2_1_1,-6.710608,-0.322960,-4.942422,-1.759467,-2.102650,-8.752433,-5.245772,-13.760251,-14.242679,-13.089189
depenses_ht_poste_01_1_2_3_1,4.409781,-2.408783,-1.158882,2.336762,1.223734,-2.411018,0.987517,2.967029,0.608847,1.638490
depenses_ht_poste_01_1_2_4_1,-3.466354,1.241681,0.901371,4.065182,1.090511,1.880281,3.952913,2.356584,1.677285,1.543249
depenses_ht_poste_01_1_2_4_2,3.308667,0.094530,-8.102628,4.600916,-4.006742,-2.505797,-10.422070,-1.778730,-5.480905,-5.076636
depenses_ht_poste_01_1_2_5_1,-6.208864,-12.778656,-4.009220,-3.482619,0.479702,-2.670528,-5.945080,-1.112958,-4.835434,-3.046017
